In [1]:
import pandas as pd
from datetime import datetime, timedelta

base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

# Generar fechas de febrero 2020
dates = pd.date_range(start='2020-02-01', end='2020-02-29', freq='D')

dfs = []
for date in dates:
    try:
        # Formato: MM-DD-YYYY
        filename = date.strftime('%m-%d-%Y') + '.csv'
        url = base_url + filename
        print(f"Cargando: {filename}")
        
        df = pd.read_csv(url)
        df['Date'] = date  # Agregar columna de fecha
        dfs.append(df)
    except Exception as e:
        print(f"Error con {filename}: {e}")

# Concatenar todos los DataFrames
df_enero = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(df_enero)}")
print(f"Fechas cargadas: {df_enero['Date'].min()} a {df_enero['Date'].max()}")

Cargando: 02-01-2020.csv
Cargando: 02-02-2020.csv
Error con 02-02-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-03-2020.csv
Error con 02-03-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-04-2020.csv
Error con 02-04-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-05-2020.csv
Error con 02-05-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-06-2020.csv
Error con 02-06-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-07-2020.csv
Cargando: 02-08-2020.csv
Error con 02-08-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-09-2020.csv
Error con 02-09-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-10-2020.csv
Cargando: 02-11-2020.csv
Error con 02-11-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-12-2020.csv
Error con 02-12-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-13-2020.csv
Error con 02-13-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-14-2020.csv
Error con 02-14-2020.csv: HTTP Error 429: Too Many

In [3]:
df_enero.sample(5, random_state=42)


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date
422,England,United Kingdom,2020-02-17T23:13:06,0,2,0,2020-02-17
75,Negeri Sembilan,Malaysia,2/1/2020 11:53,0,0,0,2020-02-01
177,Cook Islands,New Zealand,2020-02-07T23:43:02,0,0,0,2020-02-07
30,Jilin,Mainland China,2/1/2020 1:52,17,0,1,2020-02-01
361,NaN,United Arab Emirates,2020-02-16T20:43:02,9,0,4,2020-02-17


In [5]:
#2. Mostrar el número total de filas y columnas del DataFrame.
print(f"   Total de filas: {df_enero.shape[0]:,}")
print(f"   Total de columnas: {df_enero.shape[1]}")

   Total de filas: 430
   Total de columnas: 7


In [ ]:
# 3. Mostrar información general del DataFrame incluyendo tipos de datos
df_enero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  318 non-null    object        
 1   Country/Region  430 non-null    object        
 2   Last Update     430 non-null    object        
 3   Confirmed       430 non-null    int64         
 4   Deaths          430 non-null    int64         
 5   Recovered       430 non-null    int64         
 6   Date            430 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 23.6+ KB


In [ ]:
#convertir las columnas necesarias (por ejemplo,fechas).
if 'Last_Update' in df_enero.columns:
    df_enero['Last_Update'] = pd.to_datetime(df_enero['Last_Update'], format='mixed', errors='coerce')
elif 'Last Update' in df_enero.columns:
    df_enero['Last Update'] = pd.to_datetime(df_enero['Last Update'], format='mixed', errors='coerce')

numeric_columns = ['Confirmed', 'Deaths', 'Recovered']
for col in numeric_columns:
    if col in df_enero.columns:
        df_enero[col] = pd.to_numeric(df_enero[col], errors='coerce').fillna(0).astype(int)

df_enero.dtypes

Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Date              datetime64[ns]
dtype: object

In [11]:
#4. Detectar y mostrar valores nulos o faltantes por columna
df_enero.isnull().sum()

Province/State    112
Country/Region      0
Last Update         0
Confirmed           0
Deaths              0
Recovered           0
Date                0
dtype: int64

In [12]:
#5. Eliminar columnas irrelevantes
columns_to_drop = ['FIPS', 'Admin2', 'Lat', 'Long_', 'Latitude', 'Longitude', 'Combined_Key']
df_enero = df_enero.drop(columns=[col for col in columns_to_drop if col in df_enero.columns])
df_enero.columns

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'Date'],
      dtype='object')

In [13]:
#6. Estandarizar nombres de columnas (usar formato snake_case)
df_enero.columns = df_enero.columns.str.lower().str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')
df_enero.columns

Index(['province_state', 'country_region', 'last_update', 'confirmed',
       'deaths', 'recovered', 'date'],
      dtype='object')

In [14]:
#7. Homogeneizar nombres de países
country_mapping = {
    'US': 'United States',
    'UK': 'United Kingdom',
    'Korea, South': 'South Korea',
    'Taiwan*': 'Taiwan',
    'Mainland China': 'China'
}

if 'country_region' in df_enero.columns:
    df_enero['country_region'] = df_enero['country_region'].replace(country_mapping)
elif 'country/region' in df_enero.columns:
    df_enero['country/region'] = df_enero['country/region'].replace(country_mapping)

df_enero['country_region'].value_counts().head(10)

country_region
China             128
Malaysia           68
United States      36
United Kingdom     32
Australia          16
Canada             11
New Zealand         8
Japan               4
Thailand            4
Macau               4
Name: count, dtype: int64

In [15]:
#8. Convertir la columna Last_Update al formato YYYY-MM-DD
if 'last_update' in df_enero.columns:
    df_enero['last_update'] = pd.to_datetime(df_enero['last_update'], format='mixed', errors='coerce').dt.strftime('%Y-%m-%d')

df_enero[['last_update']].head()

,last_update
0,2020-02-01
1,2020-02-01
2,2020-02-01
3,2020-02-01
4,2020-02-01


In [16]:
#9. Crear una columna active_cases = Confirmed - Deaths - Recovered
df_enero['active_cases'] = df_enero['confirmed'] - df_enero['deaths'] - df_enero['recovered']
df_enero[['confirmed', 'deaths', 'recovered', 'active_cases']].head()

,confirmed,deaths,recovered,active_cases
0,7153,249,168,6736
1,599,0,21,578
2,535,0,14,521
3,422,2,3,417
4,389,0,8,381


In [18]:
#10. Guardar el DataFrame limpio como covid_clean_enero2020.csv e indicar su tamaño en MB
import os

output_path = '../data/processed/covid_clean_febrero2020.csv'
df_enero.to_csv(output_path, index=False)

file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Archivo guardado: {output_path}")
print(f"Tamaño del archivo: {file_size_mb:.2f} MB")

Archivo guardado: ../data/processed/covid_clean_febrero2020.csv
Tamaño del archivo: 0.02 MB
